This example shows how to convert yolov5 pretrained weights from Pytorch to the format that can be loaded by C++ based program, and used by the DLmodelzoo action set as well as dlpy `mzmodel()`. Pytorch 1.7.1 is required to install on the client to run this function. Other versions of Pytorch may work as well, but those versions are not tested. 

In [1]:
import torch
import sys

In [2]:
def convert_yolov5_model_weights(cpp_model_path, python_model_path, imsize=(640, 480)):
    '''
    Converts a pretrained model in Python into model weights that can be loaded by C++ programs,
    and used by the MZModel() class. To run this function, the DLPy package is not required.

    Parameters
    ----------
    python_model_path : string
        Specifies the directory that contains the Python based model weights.
    cpp_model_path : string
        Specifies the derectory to store the C++ based model weights.
    imsize : tuple
        Specifies the dimension of the image passed to the trace function.
    '''
    try:
        device = torch.device('cpu')
        ckpt = torch.load(python_model_path, map_location=device)
    except:
        raise Exception('Something is wrong while loading the pretrained weights.')

    print('NOTE: Pretrained weights loaded successfully.')

    model = []
    model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().eval())
    model = model[-1]

    imsize_md = max(imsize[0], imsize[1])
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imsize_md, imsize_md).to(device).type_as(next(model.parameters())))

    img = torch.rand([1, 3, imsize[0], imsize[1]]).to(device).float()

    try:
        pred = model(img, augment=False)[0]
        cpp_model = torch.jit.trace(model, img)
        cpp_model.save(cpp_model_path)
    except:
        raise Exception('Something is wrong while saving the pretrained weights.')

    print('NOTE: Pretrained weights saved successfully.')

Variable `yolov5_src_dir` contains the top level directory for the yolov5 source code by ultralytics, v5.0. Other versions of the model from the same repository of ultralytics may not be compatible. The requirement comes from the fact that the python model is saved in a format that depends on the source code directory. To download the source code, you can use the following command:
1. git clone https://github.com/ultralytics/yolov5.git
2. cd yolov5
3. git checkout v5.0

In [3]:
yolov5_src_dir = "/path/to/yolov5"    
sys.path.insert (0, yolov5_src_dir)

Variable `python_model_path` contains the file name for the model weights. The weights need to be compatible with DLmodelzoo yolov5 models, i.e., the yolov5 models by ultralytics, v5.0. You can download the weights from the following links:    
- https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5m.pt
- https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5s.pt
- https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5l.pt
- https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5x.pt

In [4]:
python_model_path = "/path/to/yolov5s.pt"   
cpp_model_path = "/path/to/yolov5s_cpp.pt"  

In [5]:
convert_yolov5_model_weights(cpp_model_path, python_model_path)

NOTE: Pretrained weights loaded successfully.
NOTE: Pretrained weights saved successfully.


In [6]:
import swat as sw
from dlpy.mzmodel import *
from dlpy.utils import *

In [7]:
# Connect to an existing CAS server
s = sw.CAS('your-server.unx.company.com',2999)

In [8]:
s.table.addcaslib(activeonadd=False,
                  datasource={'srctype':'path'},
                  name='cas_shlong',
                  path='/example/path',
                  subdirectories=True)

NOTE: Cloud Analytic Services added the caslib 'cas_shlong'.


,Name,Type,Description,Path,Definition,Local,Active,Personal,Hidden,Transient
0,cas_shlong,PATH,,/example/path/,,1.0,0.0,0.0,0.0,0.0


In [10]:
s.setsessopt(caslib='cas_shlong')

NOTE: 'cas_shlong' is now the active caslib.


+ Elapsed: 0.00201s, user: 0.00191s, sys: 4.5e-05s, mem: 0.273mb

In [11]:
s.table.loadTable(
        caslib="cas_shlong",
        path="coco128/obj_table.txt",
        casout=dict(name="objdet_train128", blocksize="1", replace=True),
    )  

NOTE: Cloud Analytic Services made the file coco128/obj_table.txt available as table OBJDET_TRAIN128 in caslib cas_shlong.


[caslib]

 'cas_shlong'

[tableName]

 'OBJDET_TRAIN128'

[casTable]

 CASTable('OBJDET_TRAIN128', caslib='cas_shlong')

+ Elapsed: 0.199s, user: 0.239s, sys: 0.12s, mem: 82.9mb

In [12]:
model = MZModel(conn=s, model_type="torchNative", model_name="yolov5", model_subtype="small", num_classes=80, 
                dataset_type= "OBJDETECT", anchors="10 13 16 30 33 23 30 61 62 45 59 119 116 90 156 198 373 326",
                model_path="/path/to/yolov5s_cpp.pt")

In [13]:
model.add_image_transformation(image_size='640', image_resize_type="RETAIN_ASPECTRATIO")

In [14]:
optimizer=Optimizer(seed=54321, 
                    algorithm=SGDSolver(lr=0.001, momentum=0.9),
                    batch_size=4,
                    max_epochs=5                   
                    )

In [15]:
model.train(table="objdet_train128", inputs="img_path", targets="label_path", gpu=[0], optimizer=optimizer)

NOTE: No target map from nominal values to numeric values is found.
NOTE: Using this GPU List on dl2065.clstr.rnd.sas.com: [0 ].
NOTE: dlx executable command : /r/ge.unx.sas.com/vol/vol610/u61/xingdu/pp/two_process/tkext/com/laxnd/dlxexe -controlpid 93834 -session 1
NOTE: Worker rank 0: dl2065.clstr.rnd.sas.com, using up to 1 GPU devices.
NOTE: Using dl2065.clstr.rnd.sas.com: GPU 0 processes up to 4 records at a time.
NOTE: Starting dlxexe process on worker dl2065.clstr.rnd.sas.com, rank/threadID [0/0], pid= 70489 device= GPU:0
NOTE: Learning Rate:   0.001 Momentum:     0.9 Weight Decay:       0 Dampening:       0 BatchSize: 4 Rank: 0 Device: 0 Loss: 8.43585
NOTE: The action completed successfully.


[TrainIterHistory]

    Epoch       Loss  Box Loss  Object Loss  Class Loss  Batch Loss
 0    0.0  15.510421  5.556897     5.780026    4.173499   15.510420
 1    1.0  11.813688  4.721935     4.960478    2.131276   11.813688
 2    2.0  10.088338  4.238508     4.433109    1.416722   10.088336
 3    3.0   9.072514  3.917948     4.050517    1.104049    9.072515
 4    4.0   8.435854  3.781237     3.724879    0.929738    8.435853

[TuningHistory]

 Empty SASDataFrame
 Columns: [Tuning Iteration, Number of Evaluations, Best Objective]
 Index: []

+ Elapsed: 19.3s, user: 0.114s, sys: 0.144s, mem: 91.1mb

In [16]:
model.score(table="OBJDET_TRAIN128", inputs="img_path", targets="label_path", gpu=[1], batch_size=128)

NOTE: Using this GPU List on your-server.unx.company.com: [1 ].
NOTE: No target map from nominal values to numeric values is found.
NOTE: In the score output, expected nominal values will be in raw numeric representation.
NOTE: Worker rank 0: your-server.unx.company.com, using up to 1 GPU devices.
NOTE: Using your-server.unx.company.com: GPU 1 processes up to 128 records at a time.
NOTE: Using existing dlxexe process on worker your-server.unx.company.com, rank/threadID [0/0], pid= 70489 device= GPU:1
NOTE: The action completed successfully.


,Loss,Box Loss,Object Loss,Class Loss,Batch Loss
0,9.333842,4.360626,3.977817,0.9954,9.333842
